In [3]:
from PIL import Image
import os
import torch
import numpy as np

In [ ]:
image_dir = "/content/drive/MyDrive/Data/Subset_128x128"

In [11]:
eeg_data = torch.load(r"EpochHunters/eeg_signals_raw_with_mean_std.pth")

C:\Users\CSE IIT BHILAI\AppData\Local\Temp\ipykernel_2580\3047473225.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eeg_data = torch.load(r"EpochHunters/eeg_signals_raw

In [11]:
eeg_data = torch.load(r"EpochHunters/eeg_signals_raw_with_mean_std.pth")

C:\Users\CSE IIT BHILAI\AppData\Local\Temp\ipykernel_2580\3047473225.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eeg_data = torch.load(r"EpochHunters/eeg_signals_raw

In [8]:
import os

current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

Current working directory: C:\Users\CSE IIT BHILAI


In [ ]:
eeg_data.keys()

dict_keys(['dataset', 'labels', 'images'])

In [ ]:
eeg_data["dataset"][0].keys()

dict_keys(['eeg', 'image', 'label', 'subject'])

In [ ]:
type(eeg_data["dataset"][0]['eeg'])

torch.Tensor

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

class EEGImageDataset(Dataset):
    def __init__(self, eeg_data, image_folder, synset_csv, transform=None, eeg_length=440):
        """
        Args:
            eeg_data (dict): Dictionary with keys `dataset`, `labels`, `images`.
            image_folder (str): Path to the main image folder.
            synset_csv (str): Path to the synset CSV file with `images` and `folder` columns.
            transform (callable, optional): Optional transform to be applied on the images.
            eeg_length (int): Fixed length for the EEG data (e.g., 440).
        """
        self.eeg_data = eeg_data["dataset"]
        self.image_folder = image_folder
        self.synset_df = pd.read_csv(synset_csv)
        self.transform = transform
        self.eeg_length = eeg_length

    def __len__(self):
        return len(self.eeg_data)

    def __getitem__(self, idx):
        
        eeg_entry = self.eeg_data[idx]
        eeg_signal = eeg_entry["eeg"].to(torch.float32)  # Shape: (128, variable-length)

        
        if eeg_signal.shape[1] < self.eeg_length:  
            pad_size = self.eeg_length - eeg_signal.shape[1]
            eeg_signal = F.pad(eeg_signal, (0, pad_size), mode='constant', value=0)
        elif eeg_signal.shape[1] > self.eeg_length:  
            eeg_signal = eeg_signal[:, :self.eeg_length]

        
        image_index = eeg_entry["image"]
        label = eeg_entry["label"]
        image_filename = self.synset_df.iloc[image_index]["images"] + ".JPEG"
        folder_name = self.synset_df.iloc[image_index]["folder"]
        image_path = os.path.join(self.image_folder, folder_name, image_filename)
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return {"eeg": eeg_signal, "image": image, "label": label}



# Define transformations for the images 
image_transforms = transforms.Compose([
    transforms.ToTensor(),  
])


image_folder_path = "/content/drive/MyDrive/Data/Subset_128x128"
synset_csv_path = "/content/drive/MyDrive/Data/synsetIds.csv"


image_transforms = transforms.Compose([
    transforms.ToTensor(),  
])

# Create the dataset
eeg_image_dataset = EEGImageDataset(
    eeg_data=eeg_data,
    image_folder=image_folder_path,
    synset_csv=synset_csv_path,
    transform=image_transforms
)

# Create a DataLoader 
data_loader = DataLoader(eeg_image_dataset, batch_size=16, shuffle=True)


for batch in data_loader:
    print("EEG shape:", batch["eeg"].shape)  # Shape: (batch_size, 128, 440)
    print("Image shape:", batch["image"].shape)  # Shape: (batch_size, 3, 128, 128)
    print("Labels:", batch["label"])  # Shape: (batch_size,)
    break


EEG shape: torch.Size([16, 128, 440])
Image shape: torch.Size([16, 3, 128, 128])
Labels: tensor([11,  3, 37, 37, 11, 18,  2, 14, 34,  2,  2, 17, 35,  5, 37, 12])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

## Using both for Classification and Generation Simultaneously.

In [ ]:
class ClassificationHead(nn.Module):
    def __init__(self, input_dim=40, num_classes=40):
        super(ClassificationHead, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return F.log_softmax(self.fc(x), dim=1)


In [ ]:
class GenerationHead(nn.Module):
    def __init__(self, input_dim=40, output_dim=128):
        super(GenerationHead, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)


In [ ]:
class EEGModel(nn.Module):
    def __init__(self, noise_dim=100, conditional_dim=128):
        super(EEGModel, self).__init__()
        # EEG Encoder
        self.lstm = nn.LSTM(input_size=128, hidden_size=128, batch_first=True)
        self.encoder_fc = nn.Linear(128, 40)  # Output of 40-dimensional latent feature

        # Classification Head
        self.classification_head = nn.Linear(40, 40)  # Map to latent to class logits

        # Generation Head (upsample )
        self.generation_head = nn.Linear(40, conditional_dim)

        # Noise dimension for generator
        self.noise_dim = noise_dim

    def forward(self, eeg):
        # Transpose EEG input to match the LSTM shape
        eeg = eeg.transpose(1, 2)  # Change the shape from the (batch_size, 128, 440) to the(batch_size, 440, 128)

        
        lstm_out, _ = self.lstm(eeg)  # LSTM outputs of the (batch_size, 440, 128)
        latent = self.encoder_fc(lstm_out[:, -1, :])  

        
        classification_output = self.classification_head(latent)

        
        conditional_output = self.generation_head(latent)

        return classification_output, conditional_output


In [ ]:
class ResBlockUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResBlockUp, self).__init__()
        # Main path
        self.block = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        )
        # Shortcut path
        self.upsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1),
            nn.Upsample(scale_factor=2, mode='nearest')
        )

    def forward(self, x):
        
        main_path = F.interpolate(x, scale_factor=2, mode='nearest')
        main_path = self.block(main_path)

        
        shortcut_path = self.upsample(x)

        
        return main_path + shortcut_path




class Generator(nn.Module):
    def __init__(self, noise_dim=100, conditional_dim=128):
        super(Generator, self).__init__()
        self.fc = nn.Linear(noise_dim + conditional_dim, 4 * 4 * 1024)
        self.resblock1 = ResBlockUp(1024, 512)
        self.resblock2 = ResBlockUp(512, 256)
        self.resblock3 = ResBlockUp(256, 128)
        self.resblock4 = ResBlockUp(128, 64)
        self.conv = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, noise, condition):
        x = torch.cat([noise, condition], dim=1)
        x = self.fc(x).view(-1, 1024, 4, 4)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)
        x = self.conv(x)
        return self.tanh(x)


class ResBlockDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResBlockDown, self).__init__()
        
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )
        
        self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2)

    def forward(self, x):
        
        main_path = self.block(x)

        
        shortcut_path = self.downsample(x)

        
        return main_path + shortcut_path



class Discriminator(nn.Module):
    def __init__(self, conditional_dim=128):
        super(Discriminator, self).__init__()
        self.resblock1 = ResBlockDown(3, 64)
        self.resblock2 = ResBlockDown(64, 128)
        self.resblock3 = ResBlockDown(128, 256)
        self.resblock4 = ResBlockDown(256, 512)
        self.resblock5 = ResBlockDown(512, 1024)
        self.resblock_final = ResBlockDown(1024, 1024)
        self.global_sum_pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, 1)
        self.embed = nn.Linear(conditional_dim, 1024)

    def forward(self, img, condition):
        x = self.resblock1(img)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)
        x = self.resblock5(x)
        x = self.resblock_final(x)
        x = self.global_sum_pooling(x).view(x.size(0), -1)
        condition_embed = self.embed(condition)
        x = x + condition_embed
        return self.fc(x)


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CUDA


In [ ]:
eeg_model = EEGModel().to(device)
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [ ]:
from tqdm import tqdm

In [ ]:
def train_model(eeg_model, generator, discriminator, data_loader, num_epochs=20, lr=0.0002):
    # Optimizers
    optimizer_eeg = torch.optim.Adam(eeg_model.parameters(), lr=lr)
    optimizer_gen = torch.optim.Adam(generator.parameters(), lr=lr)
    optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=lr)

    
    classification_loss_fn = nn.CrossEntropyLoss()
    gan_loss_fn = nn.BCEWithLogitsLoss()

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        
        batch_iterator = tqdm(data_loader, desc=f"Training Progress for Epoch {epoch + 1}")

        for batch in batch_iterator:
            eeg, images, labels = batch["eeg"].to(device), batch["image"].to(device), batch["label"].to(device)

            #  Classification Training 
            optimizer_eeg.zero_grad()
            class_output, condition = eeg_model(eeg)
            class_loss = classification_loss_fn(class_output, labels)
            class_loss.backward()
            optimizer_eeg.step()

            
            condition = condition.detach()

            # GAN Training 

            # Train Discriminator
            optimizer_disc.zero_grad()
            noise = torch.randn(eeg.size(0), 100).to(device)
            fake_images = generator(noise, condition)  # Generate fake images

            real_labels = torch.ones(eeg.size(0), 1).to(device)
            fake_labels = torch.zeros(eeg.size(0), 1).to(device)

            # Discriminator Loss
            real_loss = gan_loss_fn(discriminator(images, condition), real_labels)
            fake_loss = gan_loss_fn(discriminator(fake_images.detach(), condition), fake_labels)
            disc_loss = real_loss + fake_loss
            disc_loss.backward()
            optimizer_disc.step()

            # Train Generator
            optimizer_gen.zero_grad()
            noise = torch.randn(eeg.size(0), 100).to(device)  # Regenerate noise
            fake_images = generator(noise, condition)  # Re-generate fake images
            gen_loss = gan_loss_fn(discriminator(fake_images, condition), real_labels)
            gen_loss.backward()  # Backprop through generator
            optimizer_gen.step()

            
            batch_iterator.set_postfix({
                "Class Loss": f"{class_loss.item():.4f}",
                "Gen Loss": f"{gen_loss.item():.4f}",
                "Disc Loss": f"{disc_loss.item():.4f}"
            })

In [ ]:
train_model(eeg_model, generator, discriminator, data_loader, num_epochs=20, lr=0.0002)

Epoch 1/20


Training Progress for Epoch 1: 100%|██████████| 748/748 [05:49<00:00,  2.14it/s, Class Loss=3.6168, Gen Loss=15.8167, Disc Loss=0.0000]


Epoch 2/20


Training Progress for Epoch 2: 100%|██████████| 748/748 [03:25<00:00,  3.65it/s, Class Loss=3.3963, Gen Loss=17.7395, Disc Loss=0.0000]


Epoch 3/20


...

Training Progress for Epoch 18: 100%|██████████| 748/748 [03:22<00:00,  3.69it/s, Class Loss=2.1601, Gen Loss=25.8628, Disc Loss=0.0000]


Epoch 19/20


Training Progress for Epoch 19: 100%|██████████| 748/748 [03:22<00:00,  3.69it/s, Class Loss=2.5103, Gen Loss=25.9327, Disc Loss=0.0000]


Epoch 20/20


Training Progress for Epoch 20: 100%|██████████| 748/748 [03:22<00:00,  3.69it/s, Class Loss=2.0690, Gen Loss=26.0613, Disc Loss=0.0000]
